<a href="https://colab.research.google.com/github/mehulghub/PortfolioRiskManagement/blob/main/Portfolio_Optimization_and_Testing_on_Simulated_Scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
from collections import defaultdict
from datetime import datetime, timedelta
import plotly.graph_objects as go
import scipy.optimize as optimize
import yfinance as yf
import warnings
from datetime import date

warnings.filterwarnings("ignore")
RISK_FREE = 0.05

In [2]:
#Function to generate random cases of stress scenarios,i.e., market fluctuations

def generate_stress_scenarios(mean_returns, cov_matrix, num_scenarios):
    np.random.seed(42)
    return np.random.multivariate_normal(mean_returns, cov_matrix, num_scenarios)


In [3]:
#Calculates the required statistics: Portfolio Returns, Portfolio Volatility and Sharpe Ratio for Optimisation function

#Parameters
#weights: Portfolio weights
#returns: Log daily returns
#n_days: Number of trading days, i.e., 252

def statistics(weights, returns, n_days=252) -> np.array:

    portfolio_return = np.sum(np.dot(returns.mean(), weights.T)) * n_days
    excess_return = returns - RISK_FREE
    portfolio_volatility = np.sqrt(
        np.dot(
            weights,
            np.dot(excess_return.cov() * n_days, weights.T),
        )
    )
    return np.array(
        [
            portfolio_return,
            portfolio_volatility,
            (portfolio_return - RISK_FREE) / portfolio_volatility,
        ]
    )


In [4]:
#Minimisation method for the upper Statistics Method

def minimise_function(weights, returns) -> np.array:

    return -np.array(
        statistics(weights, returns)[2]
    )  # The maximum of f(x) is the minimum of -f(x)


In [12]:

'''A class for optimising a portfolio using data collected from Yahoo Finance

    Methods that I have defined in this class:
        - collect_data(): Collects historical price data for the specified stocks from Yahoo Finance.
        - optimise_portfolio(): Minimizes portfolio volatility subject to specified returns and stress test.
        - generate_plots(): Generates appropriate plots illustrating the optimized portfolio.
'''


class Portfolio:

    def __init__(self, stocks, start_date, end_date):
        """
        Initialize the PortfolioOptimizer object.

        Parameters:
            - stocks (list): List of stock symbols for portfolio construction.
            - start_date (str): Start date for collecting historical data.
            - end_date (str): End date for collecting historical data.
            - expected_returns (np.ndarray): Random set of returns generated for simulation.
        """
        self.stocks = stocks
        self.start_date = start_date
        self.end_date = end_date

        # These values will be stored later in the optimise_portfolio method
        self.returns = None

    def get_data_from_yahoo(self) -> pd.DataFrame:
        """
        Parameters:
        start: Start Date (yyyy-mm-dd) format
        end: End Date (yyyy-mm-dd) format
        """
        stock_data = yf.download(
                tickers= self.stocks, start=self.start_date, end=self.end_date
            )['Adj Close']


        return pd.DataFrame(stock_data)

    def calculate_returns(self) -> np.ndarray:
        """
        Calculates logarithmic returns of the historical data.
        """
        return_data = self.get_data_from_yahoo()
        return np.log(return_data / return_data.shift(1)).dropna()

    def optimise_portfolio(self) -> np.array:
        """
        Here I have optimized the weights with respect to the sharpe ratio.
        """

        returns = self.calculate_returns()
        self.returns = returns
        func = minimise_function
        constraints = {"type": "eq", "fun": lambda x: np.sum(x) - 1}
        # The weights can at the most be 1.
        bounds = tuple((0, 1) for _ in range(len(self.stocks)))
        random_weights = np.random.rand(len(self.stocks))
        random_weights /= np.sum(random_weights)
        optimum = optimize.minimize(
            fun=func,
            x0=np.array(
                random_weights
            ),  # We are randomly selecting a weight for optimisation
            args=returns,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
        )
        return optimum["x"]

    def stress_test_portfolio(self, num_scenarios=10000) -> np.ndarray:
        """
        Built this method that Stress tests by using optimized weights and generating random market scenarios.

        Parameters:
            - num_scenarios (int): Number of random scenarios to generate.

        Returns:
            - Array of portfolio statistics for each scenario.
        """

        optimised_weights = self.optimise_portfolio()
        print(optimised_weights)

        # Calculate mean and covariance matrix from historical data
        returns = self.returns
        mean_returns = returns.mean(axis=0)
        cov_matrix = np.cov(returns, rowvar=False)

        # Generate stress scenarios
        stress_scenarios = generate_stress_scenarios(
            mean_returns, cov_matrix, num_scenarios
        )

        portfolio_volatility = np.sqrt(
            np.dot(optimised_weights, np.dot(cov_matrix, optimised_weights.T))
        )
        print(portfolio_volatility)

        # Calculate portfolio statistics for each scenario
        portfolio_statistics = defaultdict(list)
        equally_weighted_portfolio_stats = defaultdict(list)
        for i in range(num_scenarios):
            scenario_returns = stress_scenarios[i]
            portfolio_return = np.sum(np.dot(scenario_returns, optimised_weights.T))
            portfolio_statistics["Return"].append(portfolio_return)
            equally_weighted_portfolio_stats["Return"].append(np.mean(scenario_returns))

        return pd.DataFrame(portfolio_statistics), pd.DataFrame(
            equally_weighted_portfolio_stats)



In [13]:
def plot_stress_test_results(test_results_mpt, test_results_equally_weighted):
    """
    Plot histograms of stress test results for two distributions.

    Parameters:
        - test_results_mpt (pd.DataFrame): DataFrame of portfolio statistics for MPT scenarios.
        - test_results_equally_weighted (pd.DataFrame): DataFrame of portfolio statistics for equally weighted scenarios.
    """
    fig = go.Figure()

    # Add histogram trace for MPT distribution
    fig.add_trace(
        go.Histogram(
            x=test_results_mpt["Return"],
            opacity=0.7,
            name="MPT",
            marker_color="blue",
        )
    )

    # Add histogram trace for equally weighted distribution
    fig.add_trace(
        go.Histogram(
            x=test_results_equally_weighted["Return"],
            opacity=0.7,
            name="Equally Weighted",
            marker_color="yellow",
        )
    )

    # Update layout
    fig.update_layout(
        title="Returns under different scenarios",
        xaxis_title="Portfolio Return",
        barmode="overlay",
    )

    fig.show()



In [14]:
if __name__ == "__main__":
    STOCKS = [
        'KOTAKBANK.NS', 'ICICIBANK.NS','5876.TW', '3988.HK', 'MUFG', 'SMFG'
    ]
    END = date(2024, 6, 16)
    START = END - timedelta(1 * 365)  # data of 1 years
    portfolio = Portfolio(stocks=STOCKS, start_date=START, end_date=END)
    mpt_result, equally_weighted_result = portfolio.stress_test_portfolio()
    plot_stress_test_results(mpt_result, equally_weighted_result)

    # Printing the mean and Sharpe ratio of the portfolio
    print(
        f"The average return of an equally weighted portfolio is {np.mean(equally_weighted_result['Return'])}"
    )
    print(
        f"The average return of an mpt-based portfolio is {np.mean(mpt_result['Return'])}"
    )

[*********************100%%**********************]  6 of 6 completed


[3.50526482e-01 3.78115508e-17 2.31748907e-01 0.00000000e+00
 4.17724611e-01 0.00000000e+00]
0.00901992424238604


The average return of an equally weighted portfolio is 0.0009051992412455869
The average return of an mpt-based portfolio is 0.0015134849634648158
